# TODO
See CSV Tables
https://docutils.sourceforge.io/docs/ref/rst/directives.html#csv-table


In [2]:
from msticpy.data import QueryProvider
import pandas as pd
from msticpy.data import data_providers
from msticpy.data.data_providers import DataEnvironment

env_q_stores = {}
for environ in ["LogAnalytics", "SecurityGraph", "MDATP", "LocalData"]:
    env_q_stores[environ] = QueryProvider(environ)
    

query_series = []
for env, env_queries in env_q_stores.items():
    query_names = env_queries.list_queries()
    for query_name in query_names:
        q_group, q_name = query_name.split(".")
        query_path = [env, q_group, q_name]
        q_obj = getattr(getattr(env_queries, q_group), q_name)
        qry = env_queries._query_store.get_query(q_group + "." +  q_name)
        if "table" in qry.default_params:
           q_table = qry.default_params["table"].get("default", "na").split()[0]
        elif "table" in qry.required_params:
            q_table = qry.required_params["table"].get("default", "na").split()[0]
        else:
            q_table = "-"
        q_dict = {
            "Environment": env,
            "QueryGroup": q_group,
            "Query": q_name,
            "Description": qry.description,
            "ReqdParams": ", ".join([f"{param} ({p_data.get('type')})" for param, p_data in qry.required_params.items()]),
            #"OtherParams": ", ".join([f"{param}" for param in qry.default_params]),
            "Table": q_table
        }
        query_series.append(pd.Series(q_dict))

query_df = pd.DataFrame(query_series)
    
# env_queries.

from tabulate import tabulate

for name, grp in query_df.groupby("Environment"):
    print(f"Queries for {name}")
    print("-" * len(f"Queries for {name}"))
    print()
    print(tabulate(grp.drop(columns="Environment"), headers="keys", showindex="never", tablefmt="rst"))
    print("\n")

file_name = "DataQueries.rst"
with open(file_name, "+w") as rst_file:
    rst_file.write("Data Queries Reference\n")
    rst_file.write("=" * len("Data Queries Reference") + "\n\n")
    
    for name, grp in query_df.groupby("Environment"):
        rst_file.write(f"Queries for {name}\n")
        rst_file.write("-" * len(f"Queries for {name}") + "\n")
        rst_file.write("\n")
        tbl_txt = tabulate(grp.drop(columns="Environment"), headers="keys", showindex="never", tablefmt="rst")
        rst_file.write(tbl_txt)
        rst_file.write("\n\n")

Queries for LocalData
---------------------

===============  ================================  ======================================  ============  =======
QueryGroup       Query                             Description                             ReqdParams    Table
===============  ================================  ======================================  ============  =======
Network          list_azure_network_flows_by_host  List Azure Network flows by host name                 -
Network          list_azure_network_flows_by_ip    List Azure Network flows by IP address                -
SecurityAlert    list_alerts                       Retrieves list of alerts                              -
WindowsSecurity  get_process_tree                  Get process tree for a process                        -
WindowsSecurity  list_host_events                  List events failures on host                          -
WindowsSecurity  list_host_logon_failures          List logon failures on host     

In [3]:
env_q_stores

{'LogAnalytics': <msticpy.data.data_providers.QueryProvider at 0x27824f28548>,
 'SecurityGraph': <msticpy.data.data_providers.QueryProvider at 0x27824f90048>,
 'MDATP': <msticpy.data.data_providers.QueryProvider at 0x2782501f848>}